### TODO: add to the requirements.txt

!pip install tweepy==3.10.0 

!pip install requests 

!pip install twython==3.9.1

In [1]:
import json
from twython import Twython, TwythonError, TwythonRateLimitError
import os
import pandas as pd

In [2]:
with open("../bearer_token.txt") as tokens:
    tokens_json = json.loads(tokens.read())
    #os.environ['BEARER_TOKEN'] = token.read()[:-1]
    os.environ['CONSUMER_KEY'] = tokens_json['consumer_key']
    os.environ['CONSUMER_SECRET'] = tokens_json['consumer_secret']
    os.environ['ACESS_TOKEN'] = tokens_json['access_token']
    os.environ['ACESS_TOKEN_SECRET'] = tokens_json['access_token_secret']

In [3]:
consumer_key = os.environ.get("CONSUMER_KEY")
consumer_secret = os.environ.get("CONSUMER_SECRET")
access_token = os.environ.get("ACCESS_TOKEN")
access_token_secret = os.environ.get("ACCESS_TOKEN_SECRET")

In [4]:
twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret) 

In [14]:
def timeline(user_id, max_id=None):
#       """ 
#   Loop over a user's timeline, starting at max_id. 
#   Generator.
#  
#   We can get up to 15 pages. 
#   This function loops up to 16 times to make the base case 
#   `len(tweets) == 0` trigger.
#   """
    for i in range(16):
        tweets = twitter.get_user_timeline(screen_name=user_id,
                                      max_id=max_id,
                                      count=200,
                                      trim_user=True,
                                      exclude_replies=True,
                                      include_rts=False,
                                      tweet_mode='extended',)
        if len(tweets) > 0:  # last page should have zero results
            for tweet in tweets:
                max_id = tweet['id'] - 1
                yield tweet
        else:
            break

In [6]:
from datetime import datetime
import time

def sleep_until(ts):
 # """ Sleep until the given UTC UNIX TIMESTAMP. """
    next_time = datetime.utcfromtimestamp(int(ts))
    now = datetime.utcnow()
    offset = (next_time - now).seconds
    print("Enhancing calm. Next try: {} (Currently {})".format(next_time, now))
    print("Sleeping for {}...".format(offset))
    time.sleep(offset)
    print("Continuing...")

In [7]:
list_members_congress_df = pd.read_csv('../data/eeuu_member_of_congress.csv', sep=';')
list_members_congress_df

,State,Member of Congress,Name,Party,Twitter,Instagram,Facebook
0,North Carolina 12th District,U.S. Representative,"Adams, Alma",D,@RepAdams,@repadams,x
1,Alabama 4th District,U.S. Representative,"Aderholt, Robert",R,@Robert_Aderholt,@robert_aderholt,x
2,California 31st District,U.S. Representative,"Aguilar, Pete",D,@RepPeteAguilar,@rep.peteaguilar,x
3,Georgia 12th District,U.S. Representative,"Allen, Rick",R,@RepRickAllen,@rep_rickallen,x
4,Texas 32nd District,U.S. Representative,"Allred, Colin",D,@RepColinAllred,@repcolinallred,x
...,...,...,...,...,...,...,...
536,Oregon,U.S. Senator,"Wyden, Ron",D,@RonWyden,@ronwyden,x
537,Kentucky 3rd District,U.S. Representative,"Yarmuth, John A.",D,@RepJohnYarmuth,NaN,x
538,Alaska At-Large,U.S. Representative,"Young, Don",R,@RepDonYoung,@repdonyoung,x
539,Indiana,U.S. Senator,"Young, Todd",R,@SenToddYoung,@sentoddyoung,x


In [8]:
list_members_congress_df = list_members_congress_df.dropna(subset = ['Party', 'Twitter'])
list_members_congress_df['Twitter'] = list_members_congress_df['Twitter'].apply(lambda x: x.replace('@', ''))

/tmp/ipykernel_4697/629263200.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list_members_congress_df['Twitter'] = list_members_congress_df['Twitter'].apply(lambda x: x.replace('@', ''))


In [25]:
fields = ['id', 'user', 'created_at', 'full_text', 'lang', 'label', 'retweet_count', 'favorite_count']
max_id = None
for index, row in list_members_congress_df.iterrows():
    df_output = pd.DataFrame(columns = fields)
    while True:
        try:
            name_user = row['Twitter']
            print(f'############# Scraping {name_user} timeline #############')
            for idx, tweet in enumerate(timeline(name_user, max_id)):
                if (idx+1) % 1000 == 0:
                    print(f'Scanned {idx+1} tweets')
                sub = {k: v for k, v in tweet.items() if k in fields}
                sub['user'] = sub['user']['id']
                sub['label'] = row['Party']
                df_output = df_output.append(sub, ignore_index=True)
                max_id = sub['id']
            max_id = None
            break
        except TwythonRateLimitError as e:
            sleep_until(e.retry_after) # sleep until the given date
        except TwythonError as e:
            # I *think* this is caused by protected profiles. 
            # I can read some user profile info, but not the timeline
            print(e)
            print("Skipping...")
            break
    df_output.rename(columns={'full_text':'text'}).to_csv(f'../data/{name_user}.csv', sep='|', encoding='utf-8', index=False)

############# Scraping RepAdams timeline #############
Scanned 1000 tweets
############# Scraping Robert_Aderholt timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepPeteAguilar timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepRickAllen timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepColinAllred timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping MarkAmodeiNV2 timeline #############
############# Scraping SenAngusKing timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepArmstrongND timeline #############
Scanned 1000 tweets
############# Scraping JodeyArrington timeline #############
############# Scraping JakeAuch timeline #############
Scanned 1000 tweets
############# Scraping RepCindyAxne timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepBrianBabin timeline #

Scanned 1000 tweets
############# Scraping GerryConnolly timeline #############
Scanned 1000 tweets
############# Scraping SenCoonsOffice timeline #############
Scanned 1000 tweets
############# Scraping repjimcooper timeline #############
Scanned 1000 tweets
############# Scraping JohnCornyn timeline #############
Scanned 1000 tweets
############# Scraping RepLouCorrea timeline #############
Scanned 1000 tweets
############# Scraping CatherineForNV timeline #############
Twitter API returned a 404 (Not Found), Sorry, that page does not exist.
Skipping...
############# Scraping RepJimCosta timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping SenTomCotton timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepJoeCourtney timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepAngieCraig timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping SenKevinCramer ti

############# Scraping RepMarkGreen timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
Scanned 3000 tweets
############# Scraping mtgreenee timeline #############
Twitter API returned a 401 (Unauthorized), An error occurred processing your request.
Skipping...
############# Scraping RepMGriffith timeline #############
Scanned 1000 tweets
############# Scraping RepRaulGrijalva timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepGrothman timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepMichaelGuest timeline #############
Scanned 1000 tweets
############# Scraping RepGuthrie timeline #############
Scanned 1000 tweets
############# Scraping RepHagedorn timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping BillHagertyTN timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepJoshHarder timeline #############
Scanned 1000 tweets
Scanned 200

############# Scraping  SenatorLeahy timeline #############
Scanned 1000 tweets
############# Scraping RepBarbaraLee timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping SenMikeLee timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepSusieLee timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepDLesko timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
Scanned 3000 tweets
############# Scraping jbletlow timeline #############
############# Scraping RepAndyLevin timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepMikeLevin timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepTedLieu timeline #############
Scanned 1000 tweets
############# Scraping RepZoeLofgren timeline #############
Scanned 1000 tweets
############# Scraping USRepLong timeline #############
############# Scraping RepLoudermilk timeli

Scanned 2000 tweets
############# Scraping SpeakerPelosi timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepGregPence timeline #############
Scanned 1000 tweets
############# Scraping RepPerlmutter timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepScottPerry timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping SenGaryPeters timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepScottPeters timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping AugustPfluger timeline #############
Scanned 1000 tweets
############# Scraping RepDeanPhillips timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping chelliepingree timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping StaceyPlaskett timeline #############
Scanned 1000 tweets
############# Scraping repmarkpocan 

Scanned 2000 tweets
############# Scraping RepBryanSteil timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepGregSteube timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepHaleyStevens timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepChrisStewart timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping StricklandforWA timeline #############
############# Scraping SenDanSullivan timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping Tom_Suozzi timeline #############
Scanned 1000 tweets
############# Scraping RepSwalwell timeline #############
Scanned 1000 tweets
############# Scraping RepMarkTakano timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping SenDuckworth timeline #############
Scanned 1000 tweets
Scanned 2000 tweets
############# Scraping RepVanTaylor timeline #############


----------------------------------------------------------------------------------------------------------